<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=16)
client

Client Scheduler: tcp://127.0.0.1:46531 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 16.47 GB


## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Ursula", "transactions": [{"transaction-id": 24228, "amount": 3706}, {"transaction-id": 54092, "amount": 3570}, {"transaction-id": 89854, "amount": 3215}, {"transaction-id": 90233, "amount": 3110}]}\n',)

In [5]:
# Requires `s3fs` library
# each partition is a remote CSV text file
import s3fs
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [6]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [7]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [8]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Ursula", "transactions": [{"transaction-id": 24228, "amount": 3706}, {"transaction-id": 54092, "amount": 3570}, {"transaction-id": 89854, "amount": 3215}, {"transaction-id": 90233, "amount": 3110}]}\n',
 '{"id": 1, "name": "George", "transactions": [{"transaction-id": 833, "amount": 13}, {"transaction-id": 1598, "amount": 13}, {"transaction-id": 1988, "amount": 13}, {"transaction-id": 2235, "amount": 13}, {"transaction-id": 2267, "amount": 14}, {"transaction-id": 2613, "amount": 13}, {"transaction-id": 2685, "amount": 14}, {"transaction-id": 3638, "amount": 14}, {"transaction-id": 3768, "amount": 13}, {"transaction-id": 4077, "amount": 13}, {"transaction-id": 4776, "amount": 13}, {"transaction-id": 5120, "amount": 15}, {"transaction-id": 5134, "amount": 13}, {"transaction-id": 5277, "amount": 14}, {"transaction-id": 6072, "amount": 12}, {"transaction-id": 7034, "amount": 14}, {"transaction-id": 7531, "amount": 13}, {"transaction-id": 8467, "amount": 14}, {"transact

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [9]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Ursula',
  'transactions': [{'transaction-id': 24228, 'amount': 3706},
   {'transaction-id': 54092, 'amount': 3570},
   {'transaction-id': 89854, 'amount': 3215},
   {'transaction-id': 90233, 'amount': 3110}]},
 {'id': 1,
  'name': 'George',
  'transactions': [{'transaction-id': 833, 'amount': 13},
   {'transaction-id': 1598, 'amount': 13},
   {'transaction-id': 1988, 'amount': 13},
   {'transaction-id': 2235, 'amount': 13},
   {'transaction-id': 2267, 'amount': 14},
   {'transaction-id': 2613, 'amount': 13},
   {'transaction-id': 2685, 'amount': 14},
   {'transaction-id': 3638, 'amount': 14},
   {'transaction-id': 3768, 'amount': 13},
   {'transaction-id': 4077, 'amount': 13},
   {'transaction-id': 4776, 'amount': 13},
   {'transaction-id': 5120, 'amount': 15},
   {'transaction-id': 5134, 'amount': 13},
   {'transaction-id': 5277, 'amount': 14},
   {'transaction-id': 6072, 'amount': 12},
   {'transaction-id': 7034, 'amount': 14},
   {'transaction-id': 7531, 'amou

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [10]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 12,
  'name': 'Alice',
  'transactions': [{'transaction-id': 2258, 'amount': 375},
   {'transaction-id': 4479, 'amount': 333},
   {'transaction-id': 7580, 'amount': 346},
   {'transaction-id': 8065, 'amount': 333},
   {'transaction-id': 9415, 'amount': 371},
   {'transaction-id': 11419, 'amount': 323},
   {'transaction-id': 13104, 'amount': 362},
   {'transaction-id': 13908, 'amount': 297},
   {'transaction-id': 13914, 'amount': 320},
   {'transaction-id': 15193, 'amount': 360},
   {'transaction-id': 16131, 'amount': 320},
   {'transaction-id': 16420, 'amount': 368},
   {'transaction-id': 17485, 'amount': 391},
   {'transaction-id': 18602, 'amount': 338},
   {'transaction-id': 19408, 'amount': 368},
   {'transaction-id': 21447, 'amount': 350},
   {'transaction-id': 22457, 'amount': 418},
   {'transaction-id': 23113, 'amount': 355},
   {'transaction-id': 26070, 'amount': 346},
   {'transaction-id': 26911, 'amount': 390},
   {'transaction-id': 27469, 'amount': 334},
   {'transact

In [11]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 79},
 {'name': 'Alice', 'count': 192},
 {'name': 'Alice', 'count': 26},
 {'name': 'Alice', 'count': 257},
 {'name': 'Alice', 'count': 334})

In [12]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(79, 192, 26, 257, 334)

In [13]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

227.3748902546093

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [14]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 2258, 'amount': 375},
  {'transaction-id': 4479, 'amount': 333},
  {'transaction-id': 7580, 'amount': 346},
  {'transaction-id': 8065, 'amount': 333},
  {'transaction-id': 9415, 'amount': 371},
  {'transaction-id': 11419, 'amount': 323},
  {'transaction-id': 13104, 'amount': 362},
  {'transaction-id': 13908, 'amount': 297},
  {'transaction-id': 13914, 'amount': 320},
  {'transaction-id': 15193, 'amount': 360},
  {'transaction-id': 16131, 'amount': 320},
  {'transaction-id': 16420, 'amount': 368},
  {'transaction-id': 17485, 'amount': 391},
  {'transaction-id': 18602, 'amount': 338},
  {'transaction-id': 19408, 'amount': 368},
  {'transaction-id': 21447, 'amount': 350},
  {'transaction-id': 22457, 'amount': 418},
  {'transaction-id': 23113, 'amount': 355},
  {'transaction-id': 26070, 'amount': 346},
  {'transaction-id': 26911, 'amount': 390},
  {'transaction-id': 27469, 'amount': 334},
  {'transaction-id': 31811, 'amount': 381},
  {'transaction-id': 31976, 'amount':

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 2258, 'amount': 375},
 {'transaction-id': 4479, 'amount': 333},
 {'transaction-id': 7580, 'amount': 346})

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(375, 333, 346)

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1325.172627229902

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [18]:
b.compute()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 67), ('Alice', 67), ('Alice', 68), ('Alice', 68), ('Alice', 68), ('Alice', 68), ('Alice', 68), ('Alice', 69), ('Alice', 69), ('Alice', 69), ('Alice', 69), ('Alice', 69), ('Alice', 69), ('Alice', 69), ('Alice', 91), ('Alice', 91), ('Bob', 43), ('Bob', 44), ('Bob', 44), ('Bob', 44), ('Bob', 44), ('Bob', 45), ('Bob', 45), ('Bob', 45), ('Bob', 57), ('Bob', 60), ('Bob', 89), ('Bob', 89), ('Bob', 89), ('Charlie', 63), ('Charlie', 63), ('Charlie', 63), ('Charlie', 63), ('Charlie', 63), ('Charlie', 63), ('Charlie', 63), ('Charlie', 63), ('Charlie', 63), ('Charlie', 84), ('Charlie', 84), ('Charlie', 126), ('Charlie', 189), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 63), ('Dan', 84), ('Dan', 84), ('Edith', 59), ('Edith', 59), ('Edith', 60), ('Edith', 60), ('Edith', 60), ('Edith', 60), ('Edith', 60), ('Edith', 60), ('Edith', 60), ('Edith', 60), ('Edith',

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 1139), ('Bob', 738), ('Charlie', 1050), ('Dan', 1050), ('Edith', 998), ('Frank', 697), ('George', 900), ('Hannah', 1052), ('Ingrid', 700), ('Jerry', 1394), ('Kevin', 1250), ('Laura', 1093), ('Michael', 850), ('Norbert', 1250), ('Oliver', 350), ('Patricia', 1300), ('Quinn', 800), ('Ray', 1300), ('Sarah', 600), ('Tim', 750), ('Ursula', 797), ('Victor', 849), ('Wendy', 1121), ('Xavier', 850), ('Yvonne', 1350), ('Zelda', 600)]
CPU times: user 415 ms, sys: 18.2 ms, total: 433 ms
Wall time: 1.24 s


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...
def amount(d):
    res = 0
    for transaction in d['transactions']:
        res += transaction['amount']
    return res

def accamount(tot, d):
    return tot + amount(d) 

result = js.foldby(key='name',
                   binop=accamount,
                   initial=0,
                   combine=add,
                   combine_initial=0).compute()

print(result)

[('Ursula', 64758026), ('George', 132139286), ('Laura', 117422419), ('Jerry', 149452018), ('Michael', 69998074), ('Edith', 63703504), ('Hannah', 279339714), ('Kevin', 306568933), ('Wendy', 780495069), ('Alice', 343193207), ('Patricia', 262001639), ('Charlie', 347978995), ('Yvonne', 193415648), ('Quinn', -22731857), ('Frank', 87119617), ('Norbert', 322186120), ('Tim', 93067232), ('Ray', 350651180), ('Sarah', 41923638), ('Dan', 284003968), ('Victor', 161042144), ('Zelda', 24810752), ('Ingrid', 103545470), ('Xavier', 45116125), ('Bob', 231100769), ('Oliver', 214457023)]


## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Ursula,"[{'transaction-id': 24228, 'amount': 3706}, {'..."
1,1,George,"[{'transaction-id': 833, 'amount': 13}, {'tran..."
2,2,Laura,"[{'transaction-id': 694, 'amount': 3980}, {'tr..."
3,3,Jerry,"[{'transaction-id': 547, 'amount': 2687}, {'tr..."
4,4,Jerry,"[{'transaction-id': 193, 'amount': 302}, {'tra..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 508 ms, sys: 45 ms, total: 553 ms
Wall time: 2.72 s


name
Alice      1139
Bob         738
Charlie    1050
Dan        1050
Edith       998
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Ursula', 'amount': 3706, 'transaction-id': 24228},
 {'id': 0, 'name': 'Ursula', 'amount': 3570, 'transaction-id': 54092},
 {'id': 0, 'name': 'Ursula', 'amount': 3215, 'transaction-id': 89854})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Ursula,3706,24228
1,0,Ursula,3570,54092
2,0,Ursula,3215,89854
3,0,Ursula,3110,90233
4,1,George,13,833


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 505 ms, sys: 60.7 ms, total: 565 ms
Wall time: 2.84 s


name
Alice       258980
Bob         119502
Charlie     198522
Dan         250397
Edith       121730
Frank       117646
George      265935
Hannah      216316
Ingrid      158437
Jerry       247114
Kevin       242010
Laura       156727
Michael     223392
Norbert     274801
Oliver       69666
Patricia    188454
Quinn       225341
Ray         342118
Sarah       173412
Tim         117519
Ursula      130377
Victor      127055
Wendy       352196
Xavier       98753
Yvonne      235779
Zelda        87821
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
